In [3]:
%%bash
pip install kaggle
mkdir ~/.kaggle
cp kaggle.json ~/.kaggle/kaggle.json
chmod 600 ~/.kaggle/kaggle.json
kaggle datasets download -d meowmeowmeowmeowmeow/gtsrb-german-traffic-sign
unzip gtsrb-german-traffic-sign.zip -d data

Defaulting to user installation because normal site-packages is not writeable
gtsrb-german-traffic-sign.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  gtsrb-german-traffic-sign.zip
  inflating: data/Meta.csv           
  inflating: data/Meta/.~lock.ClassesInformation.ods#  
  inflating: data/Meta/.~lock.ClassesInformationStrong.ods#  
  inflating: data/Meta/0.png         
  inflating: data/Meta/1.png         
  inflating: data/Meta/10.png        
  inflating: data/Meta/11.png        
  inflating: data/Meta/12.png        
  inflating: data/Meta/13.png        
  inflating: data/Meta/14.png        
  inflating: data/Meta/15.png        
  inflating: data/Meta/16.png        
  inflating: data/Meta/17.png        
  inflating: data/Meta/18.png        
  inflating: data/Meta/19.png        
  inflating: data/Meta/2.png         
  inflating: data/Meta/20.png        
  inflating: data/Meta/21.png        
  inflating: data/Meta/22.png        
  inf

CalledProcessError: Command 'b'pip install kaggle\nmkdir ~/.kaggle\ncp kaggle.json ~/.kaggle/kaggle.json\nchmod 600 ~/.kaggle/kaggle.json\nkaggle datasets download -d meowmeowmeowmeowmeow/gtsrb-german-traffic-sign\nunzip gtsrb-german-traffic-sign.zip -d data\n'' returned non-zero exit status 1.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import tensorflow as tf
from PIL import Image
import cv2
from sklearn.model_selection import train_test_split
from keras.layers import Conv2D,MaxPool2D,Dense,Flatten,Dropout
from keras.models import Sequential,load_model
from keras.utils import to_categorical

In [2]:
#list to store data and labels
dataset = []
labels = []
num_classes = 43 #total 43 classes present in the dataset

In [3]:
cur_path = os.getcwd()
print(cur_path)

C:\Users\DELL\OneDrive\Desktop\Programming\Machine Learning\Road sign detection model


IMAGE PREPROCESSING 

In [5]:
for i in range(num_classes):
    train_path = os.path.join(cur_path,'data','train',str(i))
    images = os.listdir(train_path)
    for img in images:
        try:
            cur_img = Image.open(train_path + '\\' + img)
            cur_img = cur_img.resize((30,30))
            cur_img = np.array(cur_img)
            dataset.append(cur_img)
            labels.append(i)
        except Exception as e:
            print('ERROR!')
            print(e)


KeyboardInterrupt



In [14]:
#convert list to arrays
dataset = np.array(dataset)
labels = np.array(labels)

In [15]:
print(dataset.shape)
print(labels.shape)

(39209, 30, 30, 3)
(39209,)


In [17]:
#save label and dataset for future use so that in case we want to implement them on another algo, we dont need to preprocess images again
os.mkdir('training_set')
np.save('./training_set/dataset',dataset)
np.save('./training_set/labels',labels)

In [4]:
#load the saved dataset and labels
dataset = np.load('./training_set/dataset.npy')
labels = np.load('./training_set/labels.npy')

In [5]:
X_train,X_test,y_train,y_test = train_test_split(dataset,labels,test_size = 0.2,random_state = 42)

ONE-HOT ENCODING

In [6]:
y_train = to_categorical(y_train,43)
y_test = to_categorical(y_test,43)

MODEL

In [7]:
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(5, 5), activation='relu', input_shape=X_train.shape[1:]))
model.add(Conv2D(filters=32, kernel_size=(5, 5), activation='relu'))  
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.2))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu')) 
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.2))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(rate=0.4))
model.add(Dense(43, activation='softmax'))

C:\Users\DELL\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [8]:
model.compile(loss = 'categorical_crossentropy',optimizer = 'adam' , metrics = ['accuracy'])

In [9]:
model_train = model.fit(X_train,y_train,batch_size = 32,epochs = 10,validation_data = (X_test,y_test))

Epoch 1/10
981/981 ━━━━━━━━━━━━━━━━━━━━ 79s 76ms/step - accuracy: 0.3601 - loss: 3.2311 - val_accuracy: 0.8949 - val_loss: 0.4904
Epoch 2/10
981/981 ━━━━━━━━━━━━━━━━━━━━ 79s 73ms/step - accuracy: 0.8094 - loss: 0.6522 - val_accuracy: 0.9373 - val_loss: 0.2207
Epoch 3/10
981/981 ━━━━━━━━━━━━━━━━━━━━ 80s 71ms/step - accuracy: 0.8728 - loss: 0.4384 - val_accuracy: 0.9644 - val_loss: 0.1382
Epoch 4/10
981/981 ━━━━━━━━━━━━━━━━━━━━ 71s 72ms/step - accuracy: 0.9032 - loss: 0.3392 - val_accuracy: 0.9753 - val_loss: 0.0910
Epoch 5/10
981/981 ━━━━━━━━━━━━━━━━━━━━ 66s 68ms/step - accuracy: 0.9262 - loss: 0.2526 - val_accuracy: 0.9723 - val_loss: 0.0970
Epoch 6/10
981/981 ━━━━━━━━━━━━━━━━━━━━ 81s 66ms/step - accuracy: 0.9334 - loss: 0.2352 - val_accuracy: 0.9737 - val_loss: 0.0936
Epoch 7/10
981/981 ━━━━━━━━━━━━━━━━━━━━ 90s 74ms/step - accuracy: 0.9351 - loss: 0.2197 - val_accuracy: 0.9838 - val_loss: 0.0608
Epoch 8/10
981/981 ━━━━━━━━━━━━━━━━━━━━ 75s 67ms/step - accuracy: 0.9351 - loss: 0.2334 - 

In [23]:
#data csv for label and dataset images directly taken from Test folder
def test(test_label,test_imgs):
    dataset = []
    y_test = pd.read_csv(test_label)
    label = y_test['ClassId'].values
    images = os.listdir(test_imgs)
    for img in images:
        try:
            cur_img = Image.open(test_imgs + '\\' + img)
            cur_img = cur_img.resize((30,30))
            cur_img = np.array(cur_img)
            dataset.append(cur_img)
        except Exception as e:
            print('ERROR!')
            print(e)
    X_test = np.array(dataset)
    return X_test,label

In [24]:
test_label = os.path.join(cur_path,'data','Test.csv')
test_imgs = os.path.join(cur_path,'data','Test')
X_test,label = test(test_label,test_imgs)

In [25]:
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred,axis = 1)

395/395 ━━━━━━━━━━━━━━━━━━━━ 11s 28ms/step


In [26]:
from sklearn.metrics import accuracy_score
print(accuracy_score(label,y_pred))

0.9448139350752177


In [29]:
model.save('./training_set/model.h5')